In [1]:
import numpy as np
import pandas as pd
import tclab
import time
from time import time
time1 = int (time())
del time
import time
import matplotlib.pyplot as plt

import pickle
import random
import datetime

# from tclab import TCLabModel as TCLab

# TCLab = tclab.setup(connected=False, speedup=1)


"""
Fix list

1. Start with Q1, Q2 at 0 for at least 5 min. At the begining, start with random Q1,Q2 value?
rangeIni1 = random.randrange(5,100)
rangeIni2 = random.randrange(5,100)
rangeEnd1 = random.randrange(200,400)
rangeEnd2 = random.randrange(200,400)
random_signal[0,rangeIni1:rangeEnd1] = random.randrange(1,99)
random_signal[1,rangeIni2:rangeEnd2] = random.randrange(1,99)

2. Need to save as pkl file. Working on it

3. Set the heater range, TCLab can not go over 100deC

"""


# Convert time
def convert_seconds_to_time(in_seconds):
    t1   = datetime.timedelta(seconds=in_seconds)
    days = t1.days
    _sec = t1.seconds
    (hours, minutes, seconds) = str(datetime.timedelta(seconds=_sec)).split(':')
    hours   = int(hours)
    minutes = int(minutes)
    seconds = int(seconds)
    
    result = []
    if days >= 1:
        result.append(str(days)+'d')
    if hours >= 1:
        result.append(str(hours)+'h')
    if minutes >= 1:
        result.append(str(minutes)+'m')
    if seconds >= 1:
        result.append(str(seconds)+'s')
    return ' '.join(result)


# Choose length of training data 
nstep = 15000

# Print running time
print('Running time: ' + convert_seconds_to_time(nstep) + " (" + str(nstep) + " seconds" + ")" + '\n')


# Create np array
T1 = np.zeros(nstep)
T2 = np.zeros(nstep)


# Number of Variables that need random signal
nVar = 2
random_signal = np.zeros((nVar,nstep))
rng = np.random.seed(time1) # Use time random num


# Set random signal for TCLab
for j in range(nVar):
    a_range = [0,100]
    a = np.random.rand(nstep) * (a_range[1]-a_range[0]) + a_range[0] # Range for amplitude
    a[0] = 0
    
    b_range = [200,600]
    b = np.random.rand(nstep) * (b_range[1]-b_range[0]) + b_range[0] # Range for frequency
    b = np.round(b)
    b = b.astype(int)
    b[0] = 0
    
    for i in range(1,np.size(b)):
        b[i] = b[i-1]+b[i]
    
    # Random Signal
    i=0
    while b[i]<np.size(random_signal[j]):
        k = b[i]
        random_signal[j,k:] = a[i]
        i=i+1

    
# Test variables
# print(time1)
# print(random_signal[0])
# print(random_signal[1])


# Connect to Arduino

tcLab = tclab.TCLab()
tcLab.LED(50)


#Set the initial temperature
rangeIni1 = random.randrange(100,150)
rangeIni2 = random.randrange(100,150)
rangeEnd1 = random.randrange(300,500)
rangeEnd2 = random.randrange(300,500)
random_signal[0,rangeIni1:rangeEnd1] = random.randrange(20,60)
random_signal[1,rangeIni2:rangeEnd2] = random.randrange(20,60)


# Test random range
# print (range1)
# print (range2)

start = time.time() # Save start time

# Run TCLab
for i in range(nstep):
    # Set heater values
    tcLab.Q1(random_signal[0,i])
    tcLab.Q2(random_signal[1,i])
    print('Time: ' + str(i) + \
          ' H1: ' + str(random_signal[0,i]) + \
          ' H2: ' + str(random_signal[1,i]) + \
          ' T1: ' + str(tcLab.T1)   + \
          ' T2: ' + str(tcLab.T2))

    if (tcLab.T1 > 95):
        random_temp0_time1 = random.randrange(250,350)
        random_signal[0,i:i+random_temp0_time1] = 0

    if (tcLab.T2 > 95):
        random_temp0_time2 = random.randrange(250,350)
        random_signal[1,i:i+random_temp0_time2] = 0

    # Save temp variable
    T1[i] = tcLab.T1
    T2[i] = tcLab.T2


    # Record every 1 sec
    time.sleep(1)


# Close connection to Arduino
tcLab.LED(0)
tcLab.close()



print ("Running time: ", nstep, 's')
print ("Actual time: ", time.time() - start, 's')



# Read data file
tcL_data = pd.DataFrame(
        {"H1": random_signal[0],
         "H2": random_signal[1],
         "T1": T1,
         "T2": T2},
        index = np.linspace(1,nstep,nstep,dtype=int))

tcL_data.to_pickle('TCLab_MIMO_Data_027_15000.pkl')




# Road data to check
tcL_data_out = pd.read_pickle('TCLab_MIMO_Data_027_15000.pkl')

print (tcL_data_out)




# Plot measurements
plt.figure(0) 
plt.subplot(2,1,1)
plt.plot(random_signal[0],'r-', drawstyle='steps',label='Heater 1')
plt.plot(random_signal[1],'b--', drawstyle='steps',label='Heater 2')
plt.ylabel('Heater (%)')
plt.legend(loc='best')

plt.subplot(2,1,2)
plt.plot(T1,'r-',label='T 1')
plt.plot(T2,'b--',label='T 2')
plt.ylabel('Temperature (degC)')
plt.legend(loc='best')
plt.xlabel('Time (sec)')

plt.show()
plt.savefig('TCLab_MIMO_Data_027.png')






Running time: 4h 10m (15000 seconds)

TCLab version 0.4.9
Arduino Leonardo connected on port /dev/ttyACM0 at 115200 baud.
TCLabFirmar 2..1Aruio eonaro/icr.
Time: 0 H1: 0.0 H2: 0.0 T1: 42.49 T2: 35.078
Time: 1 H1: 0.0 H2: 0.0 T1: 42.329 T2: 35.078
Time: 2 H1: 0.0 H2: 0.0 T1: 42.168 T2: 35.078
Time: 3 H1: 0.0 H2: 0.0 T1: 42.329 T2: 35.078
Time: 4 H1: 0.0 H2: 0.0 T1: 42.168 T2: 35.078


KeyboardInterrupt: 

In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(y=T1, name="T1"), row=1, col=1)
fig.add_trace(go.Scatter(y=T2, name="T2"), row=1, col=1)
fig.add_trace(go.Scatter(y=random_signal[0], name="H1"), row=1, col=1)
fig.add_trace(go.Scatter(y=random_signal[1], name="H2"), row=1, col=1)
